## **Jacobian**

Packages used: 
<span style="color:yellow; background-color:green">Symbolics, ForwardDiff</span>

In [20]:
using Symbolics, ForwardDiff

Symbolic expressions => `Symbolics.jacobian`

In [4]:
# define variables
@variables x1 x2
# another way => @variables x[1:2] , then use x[1], x[2], and [x[1], x[2]] for [x1, x2]

fa = [x1 * (x2)^2; x2*( sin(x1) )^3 + 11; 4*(x1)^2 + ( x2 +3 )^2]
#display(fa)

Ja = Symbolics.jacobian(fa, [x1, x2])
println("(a) ")
display(Ja)
println(Ja)
println(" ")

3×2 Matrix{Num}:
                    x2^2     2x1*x2
 3x2*cos(x1)*(sin(x1)^2)  sin(x1)^3
                     8x1  2(3 + x2)

(a) 
Num[x2^2 2x1*x2; 3x2*cos(x1)*(sin(x1)^2) sin(x1)^3; 8x1 2(3 + x2)]
 


Numerical functions
 => `ForwardDiff.jacobian`

In [7]:
x0 = [sqrt(7); 1.0]
# Build a numerical function from the symbolic expression
fa_num = build_function(fa, [x1,x2])
fa_num = eval(fa_num[1])
# Compute the Jacobian of the new function at the test point
Jaca = ForwardDiff.jacobian(fa_num, x0)
display(Jaca)

3×2 Matrix{Float64}:
  1.0       5.2915
 -0.597294  0.107695
 21.166     8.0

When all you want is the Jacobian at a point, then it's
easier to use ForwardDiff instead of creating a Symbolic
function, differentiating it, and finally, evaluating it.

In [21]:
F(x) = [
    exp(x[3] * sin(x[1] * x[2])), 
    log(1 + x[1]^2 + x[3]^4)
]

x0 = [1.0, 2.0, 3.0]  # Ensure `x0` contains floats

J_F = ForwardDiff.jacobian(F, x0)

2×3 Matrix{Float64}:
 -38.2038     -19.1019  13.9128
   0.0240964    0.0      1.3012

## **Gradient**

Symbolic expressions => `Symbolics.gradient`

In [12]:
# define variables
@variables x[1:2]

f = x[1] * cos(x[2])
grad_f = Symbolics.gradient(f,[x[1], x[2]])

@show f
@show grad_f

f = x[1]*cos(x[2])
grad_f = Num[cos(x[2]), -x[1]*sin(x[2])]


2-element Vector{Num}:
       cos(x[2])
 -x[1]*sin(x[2])

Numerical functions => `ForwardDiff.gradient` 

In [16]:
x0 = [2; pi/4]
f_num = build_function(f , x)
f_num = eval(f_num)

Grad_f_num= ForwardDiff.gradient(f_num, x0)
display(Grad_f_num)

2-element Vector{Float64}:
  0.7071067811865476
 -1.414213562373095

Total Derivative and Chain rule

In [17]:
# helper function
function deriv(f, x)
    return expand_derivatives.(Symbolics.Differential(x)(f))
end

deriv (generic function with 1 method)

In [18]:
# define variables
@variables x1, x2, t

f = x1*sin(x2)

# Compute the Jacobian of f with respect to x1, x2
Jac = Symbolics.jacobian([f], [x1, x2])

# Define the functions x1(t), x2(t)
x1_def = t^2
x2_def = t^3

# Compute the derivatives of x1 and x2
dx_dt = [deriv(x1_def, t), deriv(x2_def, t)]

# Total derivative of f using chain rule
df_dt_chain = Jac * dx_dt
df_dt = substitute(df_dt_chain, Dict(x1 => x1_def, x2 => x2_def))

# The result of df_dt_chain = Jac * derivatives is a 1 x 1 column vector
df_dt = Symbolics.simplify(df_dt[1],expand=true) # extract scalar
# df_dt = Symbolics.simplify.(df_dt,expand=true) # can also use broadcasting


2t*sin(t^3) + 3(t^4)*cos(t^3)